## Step 1: Install Required Libraries

In [ ]:
!pip install tensorflow opencv-python pillow

## Step 2: Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from PIL import Image
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript, display

## Step 3: Load and Prepare MNIST Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train[..., None]
x_test = x_test[..., None]

## Step 4: Build CNN Model

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(28,28,1)),
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(10,activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Step 5: Train CNN Model

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

## Step 6: Evaluate Model

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test Accuracy:', acc)

## Step 7: Save Trained Model

In [ ]:
model.save('mnist_cnn.h5')

## Step 8: Capture Image Using Webcam

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript(f'''
        async function takePhoto() {{
          const div = document.createElement('div');
          const btn = document.createElement('button');
          btn.textContent = 'Capture';
          div.appendChild(btn);
          const video = document.createElement('video');
          div.appendChild(video);
          document.body.appendChild(div);
          const stream = await navigator.mediaDevices.getUserMedia({{video:true}});
          video.srcObject = stream;
          await video.play();
          await new Promise(resolve => btn.onclick = resolve);
          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video,0,0);
          stream.getTracks().forEach(t=>t.stop());
          div.remove();
          return canvas.toDataURL('image/jpeg', {quality});
        }}
        takePhoto().then(data => {{
          google.colab.kernel.invokeFunction('notebook.savePhoto',[data],{{}});
        }});
    ''')
    output.register_callback('notebook.savePhoto', lambda data: open(filename,'wb').write(b64decode(data.split(',')[1])))
    display(js)

take_photo()

## Step 9: Predict Digit from Captured Image

In [ ]:
img = Image.open('photo.jpg').convert('L')
img = img.resize((28,28))
arr = np.array(img)
if np.mean(arr) > 127:
    arr = 255 - arr
arr = arr.astype('float32') / 255.0
arr = arr.reshape(1,28,28,1)
pred = model.predict(arr)[0]
digit = np.argmax(pred)
confidence = np.max(pred)
print('Predicted Digit:', digit)
print('Confidence:', confidence)